In [1]:
from pathlib import Path
from split_dataset import SplitDataset
import flammkuchen as fl
from skimage import io
import shutil
import numpy as np
from scipy.ndimage import affine_transform
from fimpy.pipeline.alignment import align_2p_volume

def tf_mx(shift):
    return np.array([[1,0,shift[0]],[0,1,shift[1]],[0,0,1]])

def rot_mx(theta):
    return np.array([[np.cos(theta),-np.sin(theta),0],[np.sin(theta),np.cos(theta),0],[0,0,1]])

In [2]:
path = Path(r'C:\Users\You\Documents\test')

In [ ]:
src = SplitDataset(path/'original')
align_2p_volume(src, across_planes=False, n_jobs=5)

In [3]:
src = SplitDataset(path/'original')
aligned = sorted(list((path/'aligned').glob('*.h5')))
rot_theta = fl.load(path/"rot_theta.h5")
(path/"rotated").mkdir()
shutil.copy(path/"aligned/stack_metadata.json", path/"rotated/stack_metadata.json")

for plane in range(src.shape[1]):
    theta = rot_theta[plane]
    if theta == 0:
        shutil.copy(path/"aligned/{:04d}.h5".format(plane), path/"rotated/{:04d}.h5".format(plane))
    else:
        data = src[:, plane:plane+1, :, :]
        transformed = np.empty(data.shape)
        shifts = fl.load(aligned[plane])['shifts']
        rotate = rot_mx(theta * np.pi/180)
        for i in range(data.shape[0]): 
            this = data[i,0,:,:]
            shift = shifts[i,0,:]
            affine = tf_mx((data.shape[-1]//2,)*2) @ (rotate @ tf_mx(shift - data.shape[-1]//2))
            transformed[i,0,:,:] = affine_transform(this, affine)
        fl.save(path/"rotated/{:04d}.h5".format(plane), dict(stack_4D=transformed), compression="blosc")

In [4]:
mask = io.imread(path/"mask.tif", plugin='tifffile')
transformed = np.empty(mask.shape)

for plane in range(mask.shape[0]):
    this = mask[plane, :, :]
    theta = rot_theta[plane]
    if theta == 0:
        transformed[plane,:,:] = this
    else:
        rotate = rot_mx(theta * np.pi/180)
        affine = tf_mx((mask.shape[-1]//2,)*2) @ (rotate @ tf_mx((-mask.shape[-1]//2,)*2))
        transformed[plane,:,:] = affine_transform(this, affine)
io.imsave(path/"mask_rotated.tif", transformed, plugin='tifffile')